In [1]:
import os

# Check the updated current working directory
current_directory = os.getcwd()

print(current_directory)


C:\Users\somad


In [2]:
from IPython.core.compilerop import contextmanager
!pip install pulp
import pulp
import pandas as pd

In [3]:
# Change the current working directory
new_directory = "C:/Users/somad/OneDrive/Documents/3rd semester/Scientific project/Scenario 2"
os.chdir(new_directory)
# Read the Excel file
coverage_df = pd.read_excel("Final_VFS_Coverage.xlsx", header=None)
population_df = pd.read_excel("VFS_population_weights.xlsx", header=None)

In [4]:
# Convert the DataFrames to matrices
Coverage = coverage_df.values.tolist()
population = population_df.values.flatten().tolist()


In [5]:
# Constants
num_circles = 27
radius = 1.66
distance_between_circles = 3.32
max_stations = 6

In [6]:
# Set up the problem
prob = pulp.LpProblem("Population_Coverage_Problem", pulp.LpMaximize)

# Decision variables
stations = pulp.LpVariable.dicts("Station", range(num_circles), cat="Binary")

# Objective function: maximize the total population covered
prob += pulp.lpSum(population[i] * stations[i] for i in range(num_circles))

# Constraints
# Ensure 100% coverage for the selected circles
for i in range(num_circles):
    prob += (
        pulp.lpSum(
            stations[j]
            for j in range(num_circles)
            if Coverage[i][j] == 1
        )
        >= 1
    )


In [7]:
# Limit the number of selected circles
prob += pulp.lpSum(stations) <= max_stations

# Solve the problem
prob.solve()

1

In [8]:
# Print the optimal solution
print("Optimal Solution:")
selected_circles = [i + 1 for i in range(num_circles) if pulp.value(stations[i]) == 1]
print("Selected Circles:", selected_circles)


Optimal Solution:
Selected Circles: [2, 5, 9, 13, 23, 25]


In [9]:
# Loop through different maximum number of stations
for max_stations in range(1, 7):
    # Set up the problem
    prob = pulp.LpProblem("Population_Coverage_Problem", pulp.LpMaximize)

    # Decision variables
    stations = pulp.LpVariable.dicts("Station", range(num_circles), cat="Binary")

    # Objective function: maximize the total population covered
    prob += pulp.lpSum(population[i] * stations[i] for i in range(num_circles))

    # Constraints
    # Ensure 100% coverage
    for i in range(num_circles):
        prob += (
            pulp.lpSum(
                stations[j]
                for j in range(num_circles)
                if Coverage[i][j] <= 1
            )
            >= 1
        )

    # Limit the number of selected circles
    prob += pulp.lpSum(stations) <= max_stations

    # Solve the problem
    prob.solve()

    # Print the optimal solution
    print(f"Max Stations: {max_stations}")
    selected_circles = [i + 1 for i in range(num_circles) if pulp.value(stations[i]) == 1]
    print("Selected Circles:", selected_circles)
    print("Total Population Covered:", pulp.value(prob.objective))
    print("-" * 50)


Max Stations: 1
Selected Circles: [8]
Total Population Covered: 46885.0
--------------------------------------------------
Max Stations: 2
Selected Circles: [5, 8]
Total Population Covered: 72812.0
--------------------------------------------------
Max Stations: 3
Selected Circles: [5, 8, 11]
Total Population Covered: 98548.0
--------------------------------------------------
Max Stations: 4
Selected Circles: [5, 8, 9, 11]
Total Population Covered: 123070.0
--------------------------------------------------
Max Stations: 5
Selected Circles: [5, 8, 9, 11, 14]
Total Population Covered: 145691.0
--------------------------------------------------
Max Stations: 6
Selected Circles: [5, 8, 9, 11, 14, 19]
Total Population Covered: 167058.0
--------------------------------------------------
